<a href="https://colab.research.google.com/github/nehaa28/PDFquestionAnswer/blob/main/pdf_chat.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
!pip install langchain
!pip install unstructured # The unstructured library provides open-source components for pre-processing text documents such as PDFs, HTML and Word Documents. 
!pip install openai
!pip install pybind11 # pybind11 is a lightweight header-only library that exposes C++ types in Python
!pip install chromadb # the AI-native open-source embedding database
!pip install Cython # Cython is an optimising static compiler for both the Python programming language
!pip3 install "git+https://github.com/philferriere/cocoapi.git#egg=pycocotools&subdirectory=PythonAPI" # COCO is a large image dataset designed for object detection, segmentation, person keypoints detection, stuff segmentation, and caption generation
!pip install unstructured[local-inference]
!CC=clang CXX=clang++ ARCHFLAGS="-arch x86_64" pip install 'git+https://github.com/facebookresearch/detectron2.git' # Detectron2 is Facebook AI Research's next generation library that provides state-of-the-art detection and segmentation algorithms.
!pip install layoutparser[layoutmodels,tesseract] # A Unified Toolkit for Deep Learning Based Document Image Analysis
!pip install pytesseract # Python-tesseract is an optical character recognition (OCR) tool for python.
!pip install Pillow==9.0.0 # The Python Imaging Library adds image processing capabilities to your Python interpreter. Need this version, otherwise errors occur.

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 467.5/467.5 KB 10.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.0/1.0 MB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 264.6/264.6 KB 15.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 114.2/114.2 KB 4.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 158.8/158.8 KB 9.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 49.1/49.1 KB 2.5 MB/s eta 0:00:00
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 31.3 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.0/2.0 MB 83.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 101.8/101.8 KB 12.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
  Cloning https://github.com/facebookresearch/detectron2.git to /tmp/pip-req-build-5dxwyscf
  Running command git clone --filter=blob:none --quiet https://github.com/facebookresearch/detectron2.git /tmp/pip-req-build-5dxwyscf
  Resolved https://github.com/facebookresearch/detectron2.git to commit d779ea63faa54fe42b9b4c280365eaafccb280d6
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 50.2/50.2 KB 6.4 MB/s eta 0:00:00
  Preparing metadata (setup.py) ... done
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 154.5/154.5 KB 16.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.7/1.7 MB 54.4 MB/s eta 0:00:00
  Created wheel for detectron2: filename=detectron2-0.6-cp39-cp39-linux_x86_64.whl size=2561406 sha256=728786ef33c72341cb5a5a28447615d7e6a554efd63993a2017a7ce97fbd055b
  Stored in directory: /tmp/pip-ephem-wheel-cache-fzg9ttl6/wheels/59/b4/

In [ ]:
import os
os.environ['OPENAI_API_KEY'] = 'your_api_key_here'

In [ ]:
from langchain.document_loaders import UnstructuredPDFLoader
from langchain.indexes import VectorstoreIndexCreator

In [ ]:
from detectron2.config import get_cfg
cfg = get_cfg()    
cfg.MODEL.DEVICE = 'cpu' #GPU is recommended

In [ ]:
!wget https://s21.q4cdn.com/399680738/files/doc_financials/2022/q4/Meta-12.31.2022-Exhibit-99.1-FINAL.pdf #meta earnings; replace with any pdf

--2023-04-03 15:31:59--  https://s21.q4cdn.com/399680738/files/doc_financials/2022/q4/Meta-12.31.2022-Exhibit-99.1-FINAL.pdf
Resolving s21.q4cdn.com (s21.q4cdn.com)... 185.172.149.132, 2a0b:4d07:201::1
Connecting to s21.q4cdn.com (s21.q4cdn.com)|185.172.149.132|:443... connected.
HTTP request sent, awaiting response... 200 OK
Length: 185815 (181K) [application/pdf]
Saving to: ‘Meta-12.31.2022-Exhibit-99.1-FINAL.pdf’

Meta-12.31.2022-Exh 100%[===================>] 181.46K  --.-KB/s    in 0.01s   

2023-04-03 15:31:59 (14.4 MB/s) - ‘Meta-12.31.2022-Exhibit-99.1-FINAL.pdf’ saved [185815/185815]



In [ ]:
!mkdir docs
!mv Meta-12.31.2022-Exhibit-99.1-FINAL.pdf docs

In [ ]:
text_folder = 'docs'
loaders = [UnstructuredPDFLoader(os.path.join(text_folder, fn)) for fn in os.listdir(text_folder)]

In [ ]:
!apt-get install poppler-utils # error occurs without this, pdf rendering library

Reading package lists... Done
Building dependency tree       
Reading state information... Done
The following NEW packages will be installed:
  poppler-utils
0 upgraded, 1 newly installed, 0 to remove and 23 not upgraded.
Need to get 174 kB of archives.
After this operation, 754 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu focal-updates/main amd64 poppler-utils amd64 0.86.1-0ubuntu1.1 [174 kB]
Fetched 174 kB in 1s (337 kB/s)
Selecting previously unselected package poppler-utils.
(Reading database ... 128288 files and directories currently installed.)
Preparing to unpack .../poppler-utils_0.86.1-0ubuntu1.1_amd64.deb ...
Unpacking poppler-utils (0.86.1-0ubuntu1.1) ...
Setting up poppler-utils (0.86.1-0ubuntu1.1) ...
Processing triggers for man-db (2.9.1-1) ...


In [ ]:
index = VectorstoreIndexCreator().from_loaders(loaders)

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package averaged_perceptron_tagger to
[nltk_data]     /root/nltk_data...
[nltk_data]   Unzipping taggers/averaged_perceptron_tagger.zip.


100%|██████████| 3/3 [00:00<00:00, 198.28it/s]


In [ ]:
query = "How much revenue did Meta make in 2022?"
index.query(query)

' Meta made $104.2 billion in revenue in 2022.'

In [ ]:
query = "What are meta's biggest risks?"
index.query(query)

" Meta's biggest risks include the impact of macroeconomic conditions on their business and financial results, their reliance on advertising revenue, their dependency on data signals and mobile operating systems, networks, and standards that they do not control, changes to the content or application of third-party policies that impact their advertising practices, and risks associated with new products and changes to existing products as well as other new business initiatives, including their metaverse efforts."